In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from ast import literal_eval
import pickle
import time
from torch.utils.data import DataLoader
import torch.optim as optim

from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

# === Load Frozen BigBird-Pegasus Model and Tokenizer ===
summarizer_model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
summarizer_model.eval()  # Freeze weights
for param in summarizer_model.parameters():
    param.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")



In [2]:
news_df=pd.read_csv("/teamspace/studios/this_studio/Datasets/news_min (2).tsv", sep='\t')
news_df

,News ID,Category,Topic,Headline,News body
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec..."
...,...,...,...,...,...
113757,N123757,sports,soccer_fifa_wwc,Hope who? Alyssa Naeher's penalty save sends U...,"LYON, France At the conclusion of the United..."
113758,N123758,sports,baseball_mlb,Chris Sale Explains What Specifically Has Gone...,The first half of Chris Sale's season could be...
113759,N123759,sports,basketball_nba_videos,Raptor fans jam streets to celebrate 1st NBA t...,Canadians are celebrating the country's first ...
113760,N123760,news,newspolitics,Judge won't allow Flynn to fire his attorneys,A federal judge denied the request by Michael ...


In [3]:
nid2body = dict(zip(news_df["News ID"], news_df["News body"]))

# Example check
print(f"Total items: {len(nid2body)}")
sample_nid = list(nid2body.keys())[0]
print(f"Example:\nNID: {sample_nid}\nBody: {nid2body[sample_nid][:300]}")

Total items: 113762
Example:
NID: N10000
Body: Only FIVE internationals allowed, count em, FIVE! So first off we should say, per our usual Atlanta United lineup predictions, this will be wrong. Why will it be wrong? Well, aside from the obvious, we still don't have a ton of data points from Frank de Boer in how he prefers to rotate his team for 


In [4]:
doc_text = nid2body.get('N10000', "")
doc_text

"Only FIVE internationals allowed, count em, FIVE! So first off we should say, per our usual Atlanta United lineup predictions, this will be wrong. Why will it be wrong? Well, aside from the obvious, we still don't have a ton of data points from Frank de Boer in how he prefers to rotate his team for   let's be honest   an inferior competition. We've seen how he rotates (or doesn't rotate) in CONCACAF Champions League play, but that's a bit different because CCL was clearly a priority for the club. We got one glimpse of U.S. Open Cup rotation last week when the team played a home game as the visiting team against the Charleston Battery, but will things change on the actual road against an MLS club? Here's my predicted lineup: Let's talk about it: Kann - Seems like he's the Cup keeper. Simples. CBs - I think Leandro Gonzalez Pirez is likely to be a casualty of the 5-international player limit. Miles Robinson is still young and probably isn't in need of much rest, and as far as internatio

In [5]:
# Path to your sid2text pickle file
with open('/teamspace/studios/this_studio/Codes/mappings/sid2text.pkl', 'rb') as f:
    sid2text = pickle.load(f)


In [6]:
len(sid2text)

135001

In [7]:
# === Device and Precision Setup ===
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 192

# === Utility Functions ===
def KDE_PMI(emb1, emb2, bandwidth=0.7):
    bandwidth = torch.tensor(bandwidth, dtype=torch.float32, device=emb1.device)
    diff = emb1 - emb2
    sim = torch.exp(- (diff ** 2) / (2 * bandwidth ** 2))
    return sim

def get_embedding(key, table, dim):
    if key not in table:
        table[key] = torch.nn.Parameter(torch.randn(dim, dtype=torch.float32, device=device) * 0.01, requires_grad=True)
    return table[key]


# === Load Embeddings ===
with open("../../Embeddings/summary_embeddings_pca192.pkl", "rb") as f:
    summary_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("../../Embeddings/newsbody_embeddings_pca192.pkl", "rb") as f:
    newsbody_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("../../Embeddings/headline_embeddings_pca192.pkl", "rb") as f:
    headline_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}

embed_tables = {
    'summary': summary_embed,
    'newsbody': newsbody_embed,
    'headline': headline_embed
}

# === Load Dataset ===
lookup_df = pd.read_csv("../../Datasets/Behavior_Vocab.csv").set_index('EdgeID')
train_df = pd.read_csv("../../Codes/mappings/trainfile.csv")

In [8]:
tail_set = set()

print("Building Tail2Idx from Bhist and Bpos...")
for row in tqdm(train_df.itertuples(), total=len(train_df), desc="Collecting tails"):
    try:
        bhist = literal_eval(row.Bhist)
        bpos = row.Bpos

        # Add tails from Bhist
        for b_id in bhist:
            if b_id in lookup_df.index:
                tail = lookup_df.loc[b_id, 'Tail']
                tail_set.add(tail)

        # Add tail from Bpos (in case it's not already in Bhist)
        if bpos in lookup_df.index:
            tail = lookup_df.loc[bpos, 'Tail']
            tail_set.add(tail)

    except Exception as e:
        print(f"[Skip] Error in row {row.Index}: {e}")

# === Final mappings ===
tail_ids = sorted(tail_set)
tail2idx = {tid: idx for idx, tid in enumerate(tail_ids)}
idx2tail = {idx: tid for tid, idx in tail2idx.items()}

print(f"Tail2Idx built: {len(tail2idx)} unique tail IDs.")

Building Tail2Idx from Bhist and Bpos...


Tail2Idx built: 173540 unique tail IDs.


In [29]:
class BehaviorEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_dwell = nn.Linear(1, hidden_dim, bias=False)
        self.gamma_rel = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_e = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.f = nn.Parameter(torch.empty(1, dtype=torch.float32, device=device))
        self.y_s = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_focus = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_acc = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))

        self.classifier = nn.Linear(hidden_dim, len(tail2idx))  # for tail_id classification
        self.bpos_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.init_parameters()

    def init_parameters(self):
        nn.init.kaiming_uniform_(self.W_dwell.weight, nonlinearity='linear')
        for param in [self.gamma_rel, self.y_e, self.y_s, self.y_focus, self.y_acc]:
            nn.init.normal_(param, mean=0.0, std=0.01)
        with torch.no_grad():
            self.f.copy_(torch.tensor([0.5], dtype=torch.float32, device=device))

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        total_loss = torch.tensor(0., dtype=torch.float32, device=device)
        enc_loss = torch.tensor(0., dtype=torch.float32, device=device)
        pred_loss = torch.tensor(0., dtype=torch.float32, device=device)
        embed_loss = torch.tensor(0., dtype=torch.float32, device=device)

        prev_e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)

        for t, b_id in enumerate(Bhist):
            if b_id not in lookup_df.index:
                continue

            row = lookup_df.loc[b_id]
            head_id, rel, tail_id = row['Head'], row['Relation'], row['Tail']
            dwell_s = row['dwell'] / 1300.0 if pd.notna(row['dwell']) else None

            head_emb = prev_e_b if t > 0 else embed_tables['headline'].get(head_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
            e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)

            if rel in ['click', 'skip', 'gen_summ']:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
                if rel == 'click':
                    dwell_v = self.W_dwell(torch.tensor([[dwell_s]], dtype=torch.float32, device=device)).squeeze(0) if dwell_s else torch.zeros(hidden_dim, dtype=torch.float32, device=device)
                    e_b = tail_emb + (dwell_v * self.y_e * KDE_PMI(tail_emb, head_emb))
                elif rel == 'skip':
                    e_b = self.f * tail_emb + (1. - self.f) * KDE_PMI(tail_emb, head_emb)
                elif rel == 'gen_summ':
                    hl_emb = embed_tables['headline'].get(tail_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
                    e_b = self.y_s * KDE_PMI(tail_emb, head_emb) * hl_emb

            elif rel == 'summ_gen':
                sum_emb = embed_tables['summary'].get(tail_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
                doc_emb = embed_tables['newsbody'].get(head_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
                e_b = self.y_focus * KDE_PMI(sum_emb, head_emb) + self.y_acc * KDE_PMI(sum_emb, doc_emb)
            else:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))
                e_b = tail_emb

            e_b = self.gamma_rel * e_b
            prev_e_b = e_b

            # Current step classification loss (on tail_id)
            if tail_id in tail2idx:
                logits = self.classifier(e_b.unsqueeze(0))
                target = torch.tensor([tail2idx[tail_id]], dtype=torch.long, device=device)
                enc_loss += F.cross_entropy(logits, target)

        # --- Predict e_bpos from final e_b ---
        e_b_pos_pred = self.bpos_mlp(e_b)

        # --- True e_bpos embedding ---
        b_next = lookup_df.loc[Bpos]
        tpos_id = b_next['Tail']
        #e_b_pos_true = embed_tables['newsbody'].get(tpos_id, torch.zeros(hidden_dim, dtype=torch.float32, device=device))

        # --- Classification loss over predicted e_bpos ---
        logits_pos = self.classifier(e_b_pos_pred.unsqueeze(0))
        target_pos = torch.tensor([tail2idx[tpos_id]], dtype=torch.long, device=device)
        pred_loss = F.cross_entropy(logits_pos, target_pos)
        
        # --- Combine all losses ---
        total_loss = 0.6 * enc_loss + 0.4 * pred_loss 

        return e_b, e_b_pos_pred, total_loss


In [84]:
class SummaryInverseMapper(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, e_b_pos, e_b_hist, e_doc):
        x = torch.cat([e_b_pos, e_b_hist, e_doc], dim=-1)
        return self.fc(x)


class CrossAttentionAdapter(nn.Module):
    def __init__(self, query_dim, key_value_dim, hidden_dim):
        super().__init__()
        self.query_proj = nn.Linear(query_dim, hidden_dim)
        self.key_proj = nn.Linear(key_value_dim, hidden_dim)
        self.value_proj = nn.Linear(key_value_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value):
        if query.dim() == 1: query = query.unsqueeze(0)
        if key.dim() == 1: key = key.unsqueeze(0)
        if value.dim() == 1: value = value.unsqueeze(0)

        Q = self.query_proj(query).unsqueeze(1)  # [B, 1, H]
        K = self.key_proj(key).unsqueeze(1)      # [B, 1, H]
        V = self.value_proj(value).unsqueeze(1)  # [B, 1, H]

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (Q.size(-1) ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        attended = torch.matmul(attn_weights, V)
        return self.out_proj(attended.squeeze(1))



class B2SModel(nn.Module):
    def __init__(self, encoder, inverse_mapper, adapter, summarizer_model, tokenizer, nid2body, sid2text):
        super().__init__()
        self.encoder = encoder
        self.inverse_mapper = inverse_mapper
        self.adapter = adapter
        self.summarizer = summarizer_model
        self.tokenizer = tokenizer
        self.nid2body = nid2body
        self.sid2text = sid2text
        self.adapter_to_pegasus = nn.Linear(adapter.out_proj.out_features, self.summarizer.config.d_model)


    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        # Step 1: Encoder outputs
        e_b, e_b_pos_pred, loss_enc = self.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

        # Step 2: Get doc embedding from head
        head_id = lookup_df.loc[Bpos]['Head']
        e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))

        # Step 3: Predict summary node embedding
        e_s_pos_pred = self.inverse_mapper(e_b_pos_pred, e_b, e_doc)

        # Step 4: Run summarizer (Pegasus) on doc to get generic summary
        doc_text = self.nid2body.get(head_id, "")
        inputs = self.tokenizer(doc_text, return_tensors="pt", padding=True, truncation=True).to(e_b.device)
        encoder_outputs = self.summarizer.model.encoder(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=True
        )

        with torch.no_grad():
            generic_ids = self.summarizer.generate(**inputs, max_new_tokens=10)
        generic_summary = self.tokenizer.decode(generic_ids[0], skip_special_tokens=True)

        # Step 5: Run adapter over generic summary tokens using e_s_pos_pred
        gen_inputs = self.tokenizer(generic_summary, return_tensors="pt", truncation=True).to(e_b.device)
        gen_input_embeds = self.summarizer.get_input_embeddings()(gen_inputs['input_ids'])  # [B, T, H]
        pooled = gen_input_embeds.mean(dim=1)  # [B, H]
        adapted = self.adapter(e_s_pos_pred, pooled, pooled)  # [B, H]

        # Step 6: Autoregressive generation using adapted vector as context
        generated_ids = [self.tokenizer.bos_token_id]
        for _ in range(64):
            prev_ids = torch.tensor(generated_ids, device=e_b.device).unsqueeze(0)  # [1, T]
            prev_embeds = self.summarizer.get_input_embeddings()(prev_ids)  # [1, T, H]
            attn_context = adapted.unsqueeze(1).expand(-1, prev_embeds.size(1), -1)  # [1, T, H]
            attn_context_proj = self.adapter_to_pegasus(attn_context)  # [1, T, 1024]
            final_input = prev_embeds + attn_context_proj  # [1, T, H]

            logits = self.summarizer(
                encoder_outputs=encoder_outputs,
                decoder_inputs_embeds=final_input
            ).logits


            next_token = torch.argmax(logits[:, -1, :], dim=-1).item()
            generated_ids.append(next_token)
            if next_token == self.tokenizer.eos_token_id:
                break

        pred_summary = self.tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Step 7: Supervised loss on full summary
        sid = lookup_df.loc[Bpos]['Tail']
        gold_summary = self.sid2text.get(sid, "")
        gold_inputs = self.tokenizer(gold_summary, return_tensors="pt", padding=True, truncation=True).to(e_b.device)
        labels = gold_inputs['input_ids']
        gold_embeds = self.summarizer.get_input_embeddings()(labels)
        adapted_proj = self.adapter_to_pegasus(adapted)         # [1, 1024]
        context = adapted_proj.unsqueeze(1).expand_as(gold_embeds)  # [1, 9, 1024]
        mod_inputs = gold_embeds + context

        outputs = self.summarizer(
            encoder_outputs=encoder_outputs,
            decoder_inputs_embeds=mod_inputs,
            labels=labels
        )

        loss_txt = outputs.loss

        return (loss_enc + loss_txt, loss_txt, loss_enc), pred_summary, gold_summary


In [85]:
# === Define Supporting Modules ===
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=192, key_value_dim=1024, hidden_dim=192).to(device)

encoder = BehaviorEncoder().to(device)

# === Initialize the Combined B2S Model ===
b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)


In [86]:
train_df_subset = train_df.iloc[:550].reset_index(drop=True)
train_rows = train_df_subset.iloc[:500]
test_rows = train_df_subset.iloc[500:550]



def extract_pairs(df, lookup_df, tail2idx):
    data_pairs = []
    for _, row in df.iterrows():
        try:
            Bhist = literal_eval(row['Bhist'])
            Bpos = row['Bpos']  # explicitly given next behavior
            if Bpos in lookup_df.index:
                tail_id = lookup_df.loc[Bpos, 'Tail']
                if tail_id in tail2idx:
                    data_pairs.append((Bhist, Bpos))  # use full Bhist, and correct Bpos
        except:
            continue
    return data_pairs


train_data = extract_pairs(train_rows, lookup_df, tail2idx)
test_data = extract_pairs(test_rows, lookup_df, tail2idx)

print(f"Train samples: {len(train_data)} | Test samples: {len(test_data)}")


Train samples: 500 | Test samples: 50


In [87]:
# === Pick one sample ===
sample_idx = random.randint(0, len(train_data) - 1)
Bhist, Bpos = train_data[sample_idx]

print(f"\n=== Running B2S Forward Pass ===")
print(f"Sample Index: {sample_idx}")
print(f"Behavior History Length: {len(Bhist)}")
print(f"Bpos: {Bpos}")

b2s_model.eval()
with torch.no_grad():
    e_b, e_b_pos_pred, _ = b2s_model.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

    head_id = lookup_df.loc[Bpos]['Head']
    e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))

    e_s_pos_pred = b2s_model.inverse_mapper(e_b_pos_pred, e_b, e_doc)

    # Run model to get detailed losses
    (loss_total, loss_gen, loss_enc), pred_summary, gold_summary = b2s_model(
        Bhist, Bpos, lookup_df, tail2idx, embed_tables
    )

# === Metadata ===
true_tail_id = lookup_df.loc[Bpos]['Tail']
print(f"\nPredicted e_b_pos shape: {e_b_pos_pred.shape}")
print(f"True Tail ID (Bpos): {true_tail_id}")

# === Summaries ===
print("\n--- Personalized Summary Output ---")
print(f"Gold Summary   : {gold_summary}")
print(f"Predicted      : {pred_summary}")

# === Losses ===
print("\n--- Loss Breakdown ---")
print(f"Total Loss     : {loss_total:.4f}")
print(f"Gen Loss       : {loss_gen:.4f}")
print(f"Encoder Loss   : {loss_enc:.4f}")


=== Running B2S Forward Pass ===
Sample Index: 236
Behavior History Length: 105
Bpos: B9679



Predicted e_b_pos shape: torch.Size([192])
True Tail ID (Bpos): S-238

--- Personalized Summary Output ---
Gold Summary   : The guard, a 19-year-old, told a Los Angeles television station he didn't want to press charges against Elliott
Predicted      : a : a , a [ 2020 option for [ a] , who wrote : the article [ a for a ] was reported that last game of the @xmath0-s who wrote : a last played for the article [ ] wrote : one of the players who had been selected

--- Loss Breakdown ---
Total Loss     : 775.4350
Gen Loss       : 10.2907
Encoder Loss   : 765.1443


In [ ]:
optimizer = torch.optim.Adam(b2s_model.parameters(), lr=1e-4)

num_epochs = 100
save_every = 20
checkpoint_template = "b2s_model_epoch{:03d}.pt"

for epoch in range(num_epochs):
    b2s_model.train()
    total_loss = 0.0
    total_gen_loss = 0.0

    print(f"\n🟦 Epoch {epoch+1}/{num_epochs}")
    pbar = tqdm(train_data, desc=f"Training Epoch {epoch+1}", dynamic_ncols=True)

    for Bhist, Bpos in pbar:
        optimizer.zero_grad()

        try:
            (loss_total, loss_txt, loss_enc), pred_summary, gold_summary = b2s_model(
                Bhist, Bpos, lookup_df, tail2idx, embed_tables
            )
        except Exception as e:
            pbar.set_postfix_str(f"[Skip {Bpos}] {str(e)}")
            continue

        loss_total.backward()
        optimizer.step()

        total_loss += loss_total.item()
        total_gen_loss += loss_txt.item()

        pbar.set_postfix({
            "Loss": f"{loss_total.item():.4f}",
            "GenLoss": f"{loss_txt.item():.4f}"
        })

    avg_loss = total_loss / len(train_data)
    avg_gen_loss = total_gen_loss / len(train_data)
    print(f"✅ [Epoch {epoch+1}] Avg Loss: {avg_loss:.4f} | Gen Loss: {avg_gen_loss:.4f}")

    if (epoch + 1) % save_every == 0:
        ckpt_path = checkpoint_template.format(epoch + 1)
        torch.save(b2s_model.state_dict(), ckpt_path)
        print(f"📌 Saved checkpoint: {ckpt_path}")



🟦 Epoch 1/100


Training Epoch 1: 100%|██████████| 500/500 [33:13<00:00,  3.99s/it, Loss=155.3428, GenLoss=7.0525]  


✅ [Epoch 1] Avg Loss: 567.8814 | Gen Loss: 8.8865

🟦 Epoch 2/100


Training Epoch 2: 100%|██████████| 500/500 [43:17<00:00,  5.20s/it, Loss=149.0499, GenLoss=6.6194]  


✅ [Epoch 2] Avg Loss: 545.5701 | Gen Loss: 8.0504

🟦 Epoch 3/100


...

Training Epoch 16: 100%|██████████| 500/500 [39:23<00:00,  4.73s/it, Loss=107.7720, GenLoss=2.3173] 


✅ [Epoch 16] Avg Loss: 379.4210 | Gen Loss: 4.4106

🟦 Epoch 17/100


Training Epoch 17: 100%|██████████| 500/500 [40:06<00:00,  4.81s/it, Loss=107.0946, GenLoss=2.0061] 


✅ [Epoch 17] Avg Loss: 377.4765 | Gen Loss: 4.4043

🟦 Epoch 18/100


Training Epoch 18:  53%|█████▎    | 267/500 [20:45<20:02,  5.16s/it, Loss=410.4525, GenLoss=2.3604] 

In [ ]:
# === Re-initialize your model architecture (same as training) ===
encoder = BehaviorEncoder().to(device)
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=192, key_value_dim=1024, hidden_dim=192).to(device)

b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)

# === Load weights ===
checkpoint_path = "b2s_model_epoch100.pt"  # <-- change as needed
b2s_model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print(f"✅ Loaded model checkpoint: {checkpoint_path}")


In [ ]:
import pandas as pd
from tqdm import tqdm

b2s_model.eval()
results = []

with torch.no_grad():
    for Bhist, Bpos in tqdm(test_data, desc="🔍 Evaluating"):
        try:
            loss, pred_summary, gold_summary = b2s_model(Bhist, Bpos, lookup_df, tail2idx, embed_tables)
            true_tail = lookup_df.loc[Bpos, 'Tail']
        except Exception as e:
            print(f"[Skip] {Bpos}: {e}")
            continue

        results.append({
            'Bpos': Bpos,
            'true_tail_id': true_tail,
            'pred_summary': pred_summary,
            'gold_summary': gold_summary
        })

df = pd.DataFrame(results)
df.to_csv("b2s_eval_results.csv", index=False)
print("✅ Results saved to: b2s_eval_results.csv")
